In [1]:
import torch
import sys
sys.path.insert(0, '../src')
import wandb
import pickle as pkl
import numpy as np
from torch.utils.data import DataLoader

from data_generator import SpatialDataset, train_val_test_split
from models import LinearSCI, NonlinearSCI
from trainers import Trainer

# Experimental tracking
wandb.login()

np.random.seed(2023)
torch.manual_seed(2023)
torch.cuda.manual_seed(2023)
torch.cuda.manual_seed_all(2023)
torch.backends.cudnn.deterministic = True

wandb: Currently logged in as: ziyang-jiang (carlsonlab). Use `wandb login --relogin` to force relogin


# Load data

In [2]:
with open('../data/synthetic_data.pkl', 'rb') as fp:
    data = pkl.load(fp)
neighborhood_size = data['neighborhood_size']
T = np.concatenate(
    [data['T_bar'][:,:neighborhood_size], 
     data['T'], 
     data['T_bar'][:,neighborhood_size:]], axis=1)
X, Y, s = data['X'], data['Y'], data['s']
de_0, de_1, ie_0, ie_1, te = data['de_0'], data['de_1'], data['ie_0'], data['ie_1'], data['te']

train_dataset, val_dataset, test_dataset = train_val_test_split(
    t=[T], x=X, s=s, y=Y, train_size=0.6, val_size=0.2, test_size=0.2, 
    shuffle=True, random_state=2020
)
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=50, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)
dataloaders = {'train': train_loader, 'val': val_loader, 'test': test_loader}

# Method 1: Linear model without U

### Training

In [ ]:
model = LinearSCI(
    num_interventions=1, 
    window_size=neighborhood_size*2+1, 
    confounder_dim=X.shape[1]
)

# Experimental tracking
wandb.init(
    project="deep_sci",
    name="linear_without_U", 
    allow_val_change=True
)
config = wandb.config

# Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optim = "adam"
optim_params = {
    'lr': 1e-3, 
    # 'momentum': 0.99
}
epochs, patience = 1000, 50
trainer = Trainer(
    model=model, 
    data_generators=dataloaders, 
    optim=optim, 
    optim_params=optim_params, 
    device=device,
    epochs=epochs,
    patience=patience, 
    wandb=wandb
)
trainer.train()

### Evaluation

In [4]:
y_pred_00 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=False)
y_pred_01 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=True)
y_pred_10 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=False)
y_pred_11 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=True)
de_0_pred, de_1_pred = np.mean(y_pred_10 - y_pred_00), np.mean(y_pred_11 - y_pred_01)
ie_0_pred, ie_1_pred = np.mean(y_pred_01 - y_pred_00), np.mean(y_pred_11 - y_pred_10)
te_pred = np.mean(y_pred_11 - y_pred_00)
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average local effect (T_bar = 0) = {np.abs(de_0_pred - de_0):.5f}")
print(f"Average local effect (T_bar = 1) = {np.abs(de_1_pred - de_1):.5f}")
print(f"Average interference effect (T = 0) = {np.abs(ie_0_pred - ie_0):.5f}")
print(f"Average interference effect (T = 1) = {np.abs(ie_1_pred - ie_1):.5f}")
print(f"Average total effect = {np.abs(te_pred - te):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average local effect (T_bar = 0) = 0.41407
Average local effect (T_bar = 1) = 0.41407
Average interference effect (T = 0) = 0.66975
Average interference effect (T = 1) = 0.66975
Average total effect = 0.25568


# Model 2: Linear model with U

### Training

In [ ]:
model = LinearSCI(
    num_interventions=1, 
    window_size=neighborhood_size*2+1, 
    confounder_dim=X.shape[1], 
    unobserved_confounder=True
)

# Experimental tracking
wandb.init(
    project="deep_sci",
    name="linear_with_U", 
    allow_val_change=True
)
config = wandb.config

# Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optim = "adam"
optim_params = {
    'lr': 1e-3, 
    # 'momentum': 0.99
}
epochs, patience = 1000, 50
trainer = Trainer(
    model=model, 
    data_generators=dataloaders, 
    optim=optim, 
    optim_params=optim_params, 
    device=device,
    epochs=epochs,
    patience=patience, 
    wandb=wandb
)
trainer.train()

### Evaluation

In [6]:
y_pred_00 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=False)
y_pred_01 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=True)
y_pred_10 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=False)
y_pred_11 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=True)
de_0_pred, de_1_pred = np.mean(y_pred_10 - y_pred_00), np.mean(y_pred_11 - y_pred_01)
ie_0_pred, ie_1_pred = np.mean(y_pred_01 - y_pred_00), np.mean(y_pred_11 - y_pred_10)
te_pred = np.mean(y_pred_11 - y_pred_00)
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average local effect (T_bar = 0) = {np.abs(de_0_pred - de_0):.5f}")
print(f"Average local effect (T_bar = 1) = {np.abs(de_1_pred - de_1):.5f}")
print(f"Average interference effect (T = 0) = {np.abs(ie_0_pred - ie_0):.5f}")
print(f"Average interference effect (T = 1) = {np.abs(ie_1_pred - ie_1):.5f}")
print(f"Average total effect = {np.abs(te_pred - te):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average local effect (T_bar = 0) = 0.71872
Average local effect (T_bar = 1) = 0.71872
Average interference effect (T = 0) = 1.20954
Average interference effect (T = 1) = 1.20954
Average total effect = 0.49083


# Model 3: Nonlinear model without U (f: MLP, g:MLP)

### Training

In [ ]:
model = NonlinearSCI(
    num_interventions=1, 
    window_size=neighborhood_size*2+1, 
    confounder_dim=X.shape[1], 
    f_network_type="mlp", 
    f_hidden_dims=[64,32], 
    g_network_type="mlp", 
    g_hidden_dims=[64,32]
)

# Experimental tracking
wandb.init(
    project="deep_sci",
    name="nonlinear_without_U_f_mlp_g_mlp", 
    allow_val_change=True
)
config = wandb.config

# Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optim = "adam"
optim_params = {
    'lr': 1e-3, 
    # 'momentum': 0.99
}
epochs, patience = 1000, 50
trainer = Trainer(
    model=model, 
    data_generators=dataloaders, 
    optim=optim, 
    optim_params=optim_params, 
    device=device,
    epochs=epochs,
    patience=patience, 
    wandb=wandb
)
trainer.train()

### Evaluation

In [8]:
y_pred_00 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=False)
y_pred_01 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=True)
y_pred_10 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=False)
y_pred_11 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=True)
de_0_pred, de_1_pred = np.mean(y_pred_10 - y_pred_00), np.mean(y_pred_11 - y_pred_01)
ie_0_pred, ie_1_pred = np.mean(y_pred_01 - y_pred_00), np.mean(y_pred_11 - y_pred_10)
te_pred = np.mean(y_pred_11 - y_pred_00)
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average local effect (T_bar = 0) = {np.abs(de_0_pred - de_0):.5f}")
print(f"Average local effect (T_bar = 1) = {np.abs(de_1_pred - de_1):.5f}")
print(f"Average interference effect (T = 0) = {np.abs(ie_0_pred - ie_0):.5f}")
print(f"Average interference effect (T = 1) = {np.abs(ie_1_pred - ie_1):.5f}")
print(f"Average total effect = {np.abs(te_pred - te):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average local effect (T_bar = 0) = 0.08969
Average local effect (T_bar = 1) = 0.08969
Average interference effect (T = 0) = 0.24763
Average interference effect (T = 1) = 0.24763
Average total effect = 0.15793


# Model 4: Nonlinear model with U (f: MLP, g: MLP)

### Training

In [ ]:
model = NonlinearSCI(
    num_interventions=1, 
    window_size=neighborhood_size*2+1, 
    confounder_dim=X.shape[1], 
    f_network_type="mlp", 
    f_hidden_dims=[64,32], 
    g_network_type="mlp", 
    g_hidden_dims=[64,32], 
    unobserved_confounder=True
)

# Experimental tracking
wandb.init(
    project="deep_sci",
    name="nonlinear_with_U_f_mlp_g_mlp", 
    allow_val_change=True
)
config = wandb.config

# Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optim = "adam"
optim_params = {
    'lr': 1e-3, 
    # 'momentum': 0.99
}
epochs, patience = 1000, 50
trainer = Trainer(
    model=model, 
    data_generators=dataloaders, 
    optim=optim, 
    optim_params=optim_params, 
    device=device,
    epochs=epochs,
    patience=patience, 
    wandb=wandb
)
trainer.train()

### Evaluation

In [10]:
y_pred_00 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=False)
y_pred_01 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=True)
y_pred_10 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=False)
y_pred_11 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=True)
de_0_pred, de_1_pred = np.mean(y_pred_10 - y_pred_00), np.mean(y_pred_11 - y_pred_01)
ie_0_pred, ie_1_pred = np.mean(y_pred_01 - y_pred_00), np.mean(y_pred_11 - y_pred_10)
te_pred = np.mean(y_pred_11 - y_pred_00)
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average local effect (T_bar = 0) = {np.abs(de_0_pred - de_0):.5f}")
print(f"Average local effect (T_bar = 1) = {np.abs(de_1_pred - de_1):.5f}")
print(f"Average interference effect (T = 0) = {np.abs(ie_0_pred - ie_0):.5f}")
print(f"Average interference effect (T = 1) = {np.abs(ie_1_pred - ie_1):.5f}")
print(f"Average total effect = {np.abs(te_pred - te):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average local effect (T_bar = 0) = 0.35877
Average local effect (T_bar = 1) = 0.35877
Average interference effect (T = 0) = 0.13793
Average interference effect (T = 1) = 0.13793
Average total effect = 0.22084


# Model 5: GCN model without U (f: GCN, g: MLP)

In [11]:
train_dataset, val_dataset, test_dataset = train_val_test_split(
    t=[T], x=X, s=s, y=Y, train_size=0.6, val_size=0.2, test_size=0.2, 
    shuffle=True, random_state=2020, graph_input=True
)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
dataloaders = {'train': train_loader, 'val': val_loader, 'test': test_loader}

100%|█████████████████████████████████████| 100/100 [00:00<00:00, 213668.06it/s]


### Training

In [ ]:
model = NonlinearSCI(
    num_interventions=1, 
    window_size=neighborhood_size*2+1, 
    confounder_dim=X.shape[1], 
    f_network_type="gcn", 
    g_network_type="mlp", 
    g_hidden_dims=[64,32]
)

# Experimental tracking
wandb.init(
    project="deep_sci",
    name="nonlinear_without_U_f_gcn_g_mlp", 
    allow_val_change=True
)
config = wandb.config

# Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optim = "adam"
optim_params = {
    'lr': 1e-4, 
    # 'momentum': 0.99
}
epochs, patience = 1000, 50
trainer = Trainer(
    model=model, 
    data_generators=dataloaders, 
    optim=optim, 
    optim_params=optim_params, 
    device=device,
    epochs=epochs,
    patience=patience, 
    wandb=wandb
)
trainer.train()

### Evaluation

In [13]:
y_pred_00 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=False)
y_pred_01 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=True)
y_pred_10 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=False)
y_pred_11 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=True)
de_0_pred, de_1_pred = np.mean(y_pred_10 - y_pred_00), np.mean(y_pred_11 - y_pred_01)
ie_0_pred, ie_1_pred = np.mean(y_pred_01 - y_pred_00), np.mean(y_pred_11 - y_pred_10)
te_pred = np.mean(y_pred_11 - y_pred_00)
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average local effect (T_bar = 0) = {np.abs(de_0_pred - de_0):.5f}")
print(f"Average local effect (T_bar = 1) = {np.abs(de_1_pred - de_1):.5f}")
print(f"Average interference effect (T = 0) = {np.abs(ie_0_pred - ie_0):.5f}")
print(f"Average interference effect (T = 1) = {np.abs(ie_1_pred - ie_1):.5f}")
print(f"Average total effect = {np.abs(te_pred - te):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average local effect (T_bar = 0) = 0.59919
Average local effect (T_bar = 1) = 0.59919
Average interference effect (T = 0) = 0.21057
Average interference effect (T = 1) = 0.21057
Average total effect = 0.38862


# Model 6: GCN model with U (f: GCN, g: MLP)

### Training

In [ ]:
model = NonlinearSCI(
    num_interventions=1, 
    window_size=neighborhood_size*2+1, 
    confounder_dim=X.shape[1], 
    f_network_type="gcn", 
    g_network_type="mlp", 
    g_hidden_dims=[64,32], 
    unobserved_confounder=True
)

# Experimental tracking
wandb.init(
    project="deep_sci",
    name="nonlinear_with_U_f_gcn_g_mlp", 
    allow_val_change=True
)
config = wandb.config

# Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optim = "adam"
optim_params = {
    'lr': 1e-4, 
    # 'momentum': 0.99
}
epochs, patience = 1000, 50
trainer = Trainer(
    model=model, 
    data_generators=dataloaders, 
    optim=optim, 
    optim_params=optim_params, 
    device=device,
    epochs=epochs,
    patience=patience, 
    wandb=wandb
)
trainer.train()

### Evaluation

In [15]:
y_pred_00 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=False)
y_pred_01 = trainer.predict(neighborhood_size*2+1, direct=False, indirect=True)
y_pred_10 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=False)
y_pred_11 = trainer.predict(neighborhood_size*2+1, direct=True, indirect=True)
de_0_pred, de_1_pred = np.mean(y_pred_10 - y_pred_00), np.mean(y_pred_11 - y_pred_01)
ie_0_pred, ie_1_pred = np.mean(y_pred_01 - y_pred_00), np.mean(y_pred_11 - y_pred_10)
te_pred = np.mean(y_pred_11 - y_pred_00)
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average local effect (T_bar = 0) = {np.abs(de_0_pred - de_0):.5f}")
print(f"Average local effect (T_bar = 1) = {np.abs(de_1_pred - de_1):.5f}")
print(f"Average interference effect (T = 0) = {np.abs(ie_0_pred - ie_0):.5f}")
print(f"Average interference effect (T = 1) = {np.abs(ie_1_pred - ie_1):.5f}")
print(f"Average total effect = {np.abs(te_pred - te):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average local effect (T_bar = 0) = 0.11228
Average local effect (T_bar = 1) = 0.11228
Average interference effect (T = 0) = 0.11959
Average interference effect (T = 1) = 0.11959
Average total effect = 0.23187
